### **r/place 2023 Sentiment Analysis Model Creation**

In this Jupyter notebook, I will perform transfer learning to improve a BERT model using a dataset of Reddit comments and their sentiment scores.

**Credits**<br>
* Thank you to Professor Alvin Chen of the National Taiwan Normal University for his guide to performing transfer learning with BERT using IMDB movie reviews. The tutorial can be found at: https://alvinntnu.github.io/NTNU_ENC2045_LECTURES/temp/sentiment-analysis-using-bert-keras-movie-reviews.html

**Importing Libraries**<br>
To create the NLP model, I use various libraries such as tensorflow, nltk, pandas, etc.

In [4]:
import csv
import numpy as np
import pandas as pd
import nltk
import random
import re
import sklearn
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras as keras
import transformers
import unicodedata

from transformers import BertTokenizer, TFBertForSequenceClassification

**Loading the Data**<br>
The target CSV file has Reddit comments in Column 0 and a score in Column 1. The scores correspond to the following sentiments: -1 = negative, 0 = neutral, 1 = positive.

In [17]:
# define the data path and store the comments in a list
data_path = "data/Reddit_Data.csv"
comments = []

# read the csv and store each comment with its respective score
with open(data_path, "r", encoding="utf8") as f:
    csv_reader = csv.reader(f)
    next(csv_reader)
    for row in csv_reader:
        comment, score = row
        comments.append((comment, int(score)))

# shuffle the data for variety
random.shuffle(comments)

**Splitting the Data**<br>
I'll use a split ratio of 80:20 for this model. Feel free to tweak the ratio if you'd like!

In [18]:
train_set, test_set = train_test_split(comments,
                                       test_size=0.2,
                                       random_state=24)

**Extracting X and Y**<br>
In this model, X = Reddit Comments, and Y = Score.

In [22]:
# fill in the comments and scores of the training set
x_train = []; y_train = []
for comment, score in train_set:
    x_train.append(comment)
    y_train.append(score)

# fill in the comments and scores of the testing set
x_test = []; y_test = []
for comment, score in test_set:
    x_test.append(comment)
    y_test.append(score)

('nice post however don think unfair minority appeasement has led directly the rise hindu fundamentalism but rather general hindu antagonism which tends lend itself well hindu fundamentalists for example there was post about how atrocious some aspects the rte act were about year ago remember like you mention actually discourages majority communities from running educational institutions and that for testimony sorts the genuine problem minority appeasement india does exist and ignore narrow minded stuff like that can really make the majority feel inferior and disadvantaged and that obviously will lead disenchantment and even discord that think where hindu antagonism ends hindu fundamentalism begins think when that opposition deliberate disadvantage twisted into generalized opposition the recipients the unfair advantage separate community whether nor they are adamant retaining that unfair advantage hindu fundamentalism and hindu extremists are reactionary they have played that reactionar

**Preparing Data (for the BERT Model)**<br>
Now that we have extracted the training and testing data, we can move onto converting the data so that the BERT model can take these inputs.

In [23]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
def convert_example_to_feature(review):
  return bert_tokenizer.encode_plus(review, 
                add_special_tokens = True,
                max_length = 512,
                padding='max_length',
                truncation=True,
                return_attention_mask = True,
              )